# JVS100人分で学習したモデルの音声合成デモ

モデルの読み込み

In [ ]:
from espnet2.bin.tts_inference import Text2Speech
from espnet2.train.dataset import ESPnetDataset
import os

# 実験ディレクトリのパス
exp_dir = "./exp/tts_finetune_xvector_vits_raw_phn_jaconv_pyopenjtalk_prosody"

# 学習設定ファイルのパス
train_config_path = os.path.join(exp_dir, "config.yaml")

# モデルファイルのパス
model_path = os.path.join(exp_dir, "train.total_count.ave.pth")
# model_path = os.path.join(exp_dir, "train.total_count.best.pth")
# model_path = os.path.join(exp_dir, "latest.pth")

# ボコーダモデルの名前
vocoder_tag = "parallel_wavegan/jsut_parallel_wavegan.v1"

# text2speechインスタンスの作成
text2speech = Text2Speech.from_pretrained(
    train_config=train_config_path,
    model_file=model_path,
    vocoder_tag=vocoder_tag,
)

X-Vectorとリアル音声のデータセット

In [ ]:
dataset = ESPnetDataset([
    ("./dump/22k/xvector/eval1/xvector.scp", "xvector", "kaldi_ark"),
    ("./dump/22k/raw/eval1/wav.scp", "speech", "sound")])

合成

In [ ]:
from IPython.display import display, Audio, Markdown

# i番目のデータを取得する
#   評価セットは1人あたり5データずつあるので，5つごとに個人性が変わる
i = 40 
id      = dataset[i][0][:6]         # 発話IDから個人IDを取り出す
xvector = dataset[i][1]['xvector']  # X-Vector（個人性を表すベクトル）
speech  = dataset[i][1]['speech']   # リアル音声

# 合成（spembsにxvectorを指定することで，個人性を表現する）
result = text2speech("それではこれで、明日の状況の確認をお願いしようと思います", spembs=xvector)

# 合成音の表示
display(Markdown(f"**{id}**"))
display(Markdown("- リアル"))
display(Audio(speech, rate=text2speech.fs))
display(Markdown("- 合成"))
display(Audio(result["wav"].cpu(), rate=text2speech.fs))


個人性の混合

In [ ]:
# i番目の個人性とj番目の個人性を1対1で混合する
i, j = 40, 45

id_i = dataset[i][0][:6]
xvector_i = dataset[i][1]['xvector']
speech_i = dataset[i][1]['speech']

id_j = dataset[j][0][:6]
xvector_j = dataset[j][1]['xvector']
speech_j = dataset[j][1]['speech']

text = "それではこれで、明日の状況の確認をお願いしようと思います"
# i番目の人で合成
result_i = text2speech(text, spembs=xvector_i)
# j番目の人で合成
result_j = text2speech(text, spembs=xvector_j)
# 1対1（0.5ずつの重み）で合成
w_i = 0.5
result_i_j = text2speech(text, spembs=(w_i*xvector_i+(1.0-w_i)*xvector_j))

display(Markdown(f"- リアル ({id_i})"))
display(Audio(speech_i, rate=text2speech.fs))
display(Markdown(f"- リアル ({id_j})"))
display(Audio(speech_j, rate=text2speech.fs))

display(Markdown(f"- 合成 ({id_i})"))
display(Audio(result_i["wav"].cpu(), rate=text2speech.fs))
display(Markdown(f"- 合成 ({id_j})"))
display(Audio(result_j["wav"].cpu(), rate=text2speech.fs))
display(Markdown(f"- 合成 ({id_i} + {id_j})"))
display(Audio(result_i_j["wav"].cpu(), rate=text2speech.fs))

個人性の混合（0.1ずつ重みを変えてモーフィング

In [ ]:
# i番目の人から徐々にj番目の人に移行する
i, j = 0, 15

id_i = dataset[i][0][:6]
xvector_i = dataset[i][1]['xvector']
speech_i = dataset[i][1]['speech']

id_j = dataset[j][0][:6]
xvector_j = dataset[j][1]['xvector']
speech_j = dataset[j][1]['speech']

display(Markdown(f"- リアル ({id_i})"))
display(Audio(speech_i, rate=text2speech.fs))
display(Markdown(f"- リアル ({id_j})"))
display(Audio(speech_j, rate=text2speech.fs))

# 重みを徐々に変化させて合成
import numpy as np
text = "こんにちは"
for w_j in np.arange(0.0, 1.1, 0.1):
    w_i = 1.0 - w_j
    result_i_j = text2speech(text,
                             spembs=(w_i*xvector_i+w_j*xvector_j))
    display(Markdown(f"- 合成 ({id_i}({w_i:.1f}) : {id_j}({w_j:.1f}))"))
    display(Audio(result_i_j["wav"].cpu(), rate=text2speech.fs))